In [ ]:
import { useEffect, useState } from "react";
import { useParams } from "react-router-dom";
import "./Components/styles/searchPageStyle.css";
import MovieCard from "./Components/movieCard";
import NavBar from "./Components/NavBar";
import ReactPlayer from "react-player";

const SearchPage = () => {
    const params = useParams();
    const apiKey = "api_key=7212cc714eacf62263334b404a1fc587";
    const inputValue = params.id; // retrieving the searched movie name
    const [searchedMovie, setSearchedMovie] = useState({});
    const [recommendedMovies, setRecommendedMovies] = useState([{}]);
    const [director, setDirector] = useState(""); 
    const [castMembers, setCastMembers] = useState([{}]);
    const [genreList, setGenreList] = useState([{}]);
    const [currGenre, setCurrGenre] = useState([{}]);
    const [videoData, setVideoData] = useState([]);
    const [playTrailer, setPlayTrailer] = useState(0);
    const gotCast = (castData) => {
        setCastMembers([]);

        let counter = 3;
        for (let cast of castData) {
            setCastMembers((castMembers) => [...castMembers, cast]);
            counter--;
            if (counter === 0) break;
        }
    };
    const gotVideo = (data) => {
        if (data.videos && data.videos.results) {
            // Cherche une vidéo avec la langue française
            const frenchTrailer = data.videos.results.find(
                (vid) => vid.language === "fr" && vid.name === "Official Trailer"
            );
    
            // Si une bande-annonce en français existe, utilise-la
            setVideoData(frenchTrailer ? frenchTrailer : data.videos.results[0]);
        }
    };
    
    const gotRecommendedData = (apiData) => {
        setRecommendedMovies([]);
        let counter = 8;
        // getting data for each of the recommened movies
        for (let movie of apiData.movies) {
            fetch(
                `https://api.themoviedb.org/3/search/movie?${apiKey}&query=${movie}&language=fr`
            ).then((Response) =>
                Response.json().then((data) =>
                    setRecommendedMovies((recommendedMovies) => [
                        ...recommendedMovies,
                        data.results[0],
                    ])
                )
            );
            counter--;
            if (counter === 0) break;
        }
    };
    useEffect(
        () => {
            const gotTMDBData = (apiData) => {
                const realMovieData = apiData.results[0];
                setCurrGenre([]); 
                setCurrGenre(realMovieData.genre_ids);
    
                setSearchedMovie(realMovieData);
    
                // Fetch credits to get the cast and director
                fetch(
                    `https://api.themoviedb.org/3/movie/${realMovieData.id}/credits?${apiKey}`
                ).then((Response) =>
                    Response.json().then((data) => {
                        gotCast(data.cast);
    
                        // Find the director in the crew
                        const director = data.crew.find(member => member.job === "Director");
                        if (director) {
                            setDirector(director.name); // Affiche le nom du réalisateur
                        }
                    })
                );
    
                // Fetch movie details (with videos)
                fetch(
                    `https://api.themoviedb.org/3/movie/${realMovieData.id}?${apiKey}&append_to_response=videos&language=fr`
                ).then((Response) =>
                    Response.json().then((data) => gotVideo(data))
                );
            };
    
            // Getting data for the searched movie from TMDb
            fetch(
                `https://api.themoviedb.org/3/search/movie?${apiKey}&query=${inputValue}&language=fr`
            ).then((Response) =>
                Response.json().then((data) => gotTMDBData(data))
            );
    
            // Fetching recommended movies
            fetch(`/api/similarity/${inputValue}`).then((Response) =>
                Response.json().then((data) => gotRecommendedData(data))
            );
    
            // Fetching the list of genres
            fetch(
                `https://api.themoviedb.org/3/genre/movie/list?${apiKey}&language=fr`
            ).then((Response) =>
                Response.json().then((data) => setGenreList(data.genres))
            );
        },
        [inputValue] /*Making api call whenever the searched movie changes */
    );
    
    const RenderMovies = () =>
        recommendedMovies.map((movie) => {
            if (movie) {
                return (
                    <MovieCard
                        key={movie.id + movie.original_title}
                        movie={movie}
                    />
                );
            } else {
                return null;
            }
        });
        const RenderTrailer = () => {
            return (
                <div>
                    <ReactPlayer
                        url={`https://www.youtube.com/watch?v=${videoData.key}-U`} // Lien vers la bande-annonce
                        playing={true}
                        width="100%"
                        height="100%"
                        controls={true}
                        className="youtube-container"
                    />
                </div>
            );
        };
        const displayGenre = () =>
            currGenre.map((movieId, ind) => {
                if (ind >= 3) return null;
                if (movieId) {
                    for (let obj of genreList) {
                        if (obj.id === movieId) {
                            if (ind === 2) {
                                return <span>{obj.name}</span>;
                            } else {
                                return (
                                    <span>
                                        {obj.name}
                                        {","}{" "}
                                    </span>
                                );
                            }
                        }
                    }
                } else {
                    return null;
                }
                return null;
            });
    
        const imgLink = "https://image.tmdb.org/t/p/original";
        const backdropPath = "https://image.tmdb.org/t/p/w1280";
        return (
            <div
                style={{
                    backgroundImage: `linear-gradient(to bottom, rgba(0, 0, 0, 0), rgba(0, 0, 0, 1)), url(${backdropPath}${searchedMovie.backdrop_path})`,
                }}
                className="MainBackGround"
            >
                <NavBar isHome={true} />
    
                <div className="container trailerContainer">
                    {
                        videoData && playTrailer
                            ? RenderTrailer()
                            : null /*Rendering the trailer*/
                    }
                    <div className="container .movie-details">
                        <div className="row ">
                            <div className="col-md-6 left-box col-md-push-6">
                                <h1 className="topTitle-Movie">
                                    {searchedMovie.title}{" "}
                                </h1>
                                <p className="overviewContent">
                                {searchedMovie.overview}
                                </p>
                                <p>Distribution: </p>
                                <div className="casting">
                                {/* Affiche le nom du réalisateur en premier */}
                                    {director && (
                                    <p><b>Réalisateur:</b> {director}</p>
                                    )}

                                {/* Affiche les noms des acteurs */}
                                    {castMembers.map((member) => {
                                        if (member) {
                                            return (
                                            <p key={member.cast_id + member.id}>
                                                <a
                                                    href={`https://en.wikipedia.org/wiki/${member.name}`}
                                                    target="_blank"
                                                    rel="noreferrer"
                                                >
                                                {member.name}
                                                </a>
                                            </p>
                                            );
                                        }
                                        return null;
                                    })}
                                </div>

                                <div>
                                <b>Score d'évaluation{" : "}</b>
                                {searchedMovie.vote_average}
                                {"/10 "}

                                <i className="fa-solid fa-star"></i>
                            </div>
                            <div>
                                <b> Date de Sortie</b>
                                {" : "} {searchedMovie.release_date}
                            </div>
                            <div>
                                <b>Genres</b>
                                {" : "}
                                {currGenre ? displayGenre() : null}
                            </div>
                            <div>
                                <button
                                    className="trailer-bttn "
                                    onClick={() => setPlayTrailer(true)}
                                >
                                    <i className="fa-solid fa-play"></i>
                                    {"Regarder la bande d'annonce"}
                                </button>
                            </div>
                        </div>
                        <div className="col-md-6 col-md-pull-6 text-center">
                            <img
                                className="main-img"
                                src={`https://image.tmdb.org/t/p/w500${searchedMovie.poster_path}`}
                                alt="Movie"
                            />
                        </div>
                    </div>
                </div>
            </div>
            {/*Trailer Close Button */}
            <div className={playTrailer ? "DisplayOn" : "DisplayOFF"}>
                <button
                    className="close-bttn"
                    onClick={() => setPlayTrailer(false)}
                >
                    Close Trailer
                </button>
            </div>

            <div className="container-fluid recommendedMovies">
                <h2 className=" container RecommendHeading">
                    Recommended Movies
                </h2>
                {/*Rendering the recommended movie cards */}
                <div className="container recommendedGrid">
                    {RenderMovies()}
                </div>
            </div>
        </div>
    );
};

export default SearchPage;

                                
        

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("df_nn_cosine3.csv")

In [4]:
df.columns

Index(['cat_genres', 'cat_langues', 'num_majorite_H/F', 'num_titleType',
       'num_popularity', 'num_vote_average', 'num_vote_count', 'num_runtime',
       'num_startYear', 'num_nb_film_acteur', 'num_nb_film_director',
       'num_nb_film_writer', 'opt_cat_original_title', 'opt_cat_liste_acteurs',
       'opt_cat_original_language', 'opt_cat_spoken_languages', 'Africa',
       'Asia', 'Europe', 'North America', 'Oceania', 'South America',
       'Unknown', 'cn', 'de', 'en', 'es', 'fr', 'hi', 'it', 'ja', 'ko', 'pt',
       'ru', 'zh', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller',
       'War', 'Western', '\\N', 'opt_cat_original_title.1', 'param_cosine',
       'titre_francais'],
      dtype='object')

In [61]:
# Entraînement pour NN
# Sélectionner les colonnes numériques
X = df[["num_majorite_H/F", "num_titleType", "num_popularity", "num_vote_average", "num_vote_count","num_runtime", "num_startYear",
                 'Action', 'Adventure', 'Animation', 'Biography',
                 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
                 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
                 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Sport', 'Talk-Show',
                 'Thriller', 'War', 'Western', "\\\\N", 'Africa', 'Asia', 'Europe',
                 'North America', 'Oceania', 'South America', "Unknown", 'cn', 'de', 'en',
                 'es', 'fr', 'hi', 'it', 'ja', 'ko', 'pt', 'ru', 'zh']]
# Remplir les valeurs manquantes par 0 ou une valeur pertinente
X = X.fillna(0)
# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Créer un modèle de voisins les plus proches
model_NN = NearestNeighbors(n_neighbors=1000, algorithm='auto').fit(X_scaled)


def film_nn(saisir_titre):
    saisir_titre = saisir_titre.lower()  # Convertir le titre en minuscules
    df_1000_films = {}
    # Vérifier si le titre est dans la colonne 'opt_cat_original_title'
    if saisir_titre in df['opt_cat_original_title'].values:
        try:
            # Trouver l'index du film
            film_index = df[df['opt_cat_original_title'] == saisir_titre].index[0]
            # Trouver les voisins les plus proches avec le modèle Nearest Neighbors
            distances, indices = model_NN.kneighbors([X_scaled[film_index]])
            # Stocker les voisins proches dans un dictionnaire
            for distance, index in zip(distances[0], indices[0]):
                # if index != film_index:  # Ignorer le film lui-même
                df_1000_films[df['opt_cat_original_title'].iloc[index]] = distance
            # Convertir le dictionnaire en DataFrame
            df_1000_films = pd.DataFrame.from_dict(df_1000_films, orient='index', columns=['Distance'])
            df_1000_films.reset_index(inplace=True)
            df_1000_films.rename(columns={'index': 'Film'}, inplace=True)
            # Ajouter des colonnes supplémentaires à partir de df_nn_cosine2
            df_1000_films = pd.merge(
                df_1000_films,
                df,
                how="left",
                left_on="Film",
                right_on="opt_cat_original_title"
            )
            # Calculer la Cosine Similarity sur la colonne 'param_cosine'
            vectorizer = TfidfVectorizer(stop_words="english")
            tfidf_matrix = vectorizer.fit_transform(df_1000_films['param_cosine'])
            similarity = cosine_similarity(tfidf_matrix)
            # Identifier les films les plus similaires à partir de la Cosine Similarity
            movie_index = df_1000_films[df_1000_films['opt_cat_original_title'] == saisir_titre].index[0]
            similarity_scores = list(enumerate(similarity[movie_index]))
            similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
            # Exclure le film d'origine et sélectionner les 5 films les plus proches
            similar_movies = [
                df_1000_films['opt_cat_original_title'].iloc[i[0]]
                for i in similarity_scores[1:9]
            ]
            return similar_movies
        except Exception as e:
            print(f"\nErreur lors de la recherche des voisins : {str(e)}")
            return None
    else:
        print(f"\nErreur : '{saisir_titre}' n'existe pas dans la base de données.")
        return None

In [73]:
film_nn("spirited away")


Erreur : 'spirited away' n'existe pas dans la base de données.


In [ ]:
def createSimilarity():
    data = pd.read_csv('main_data.csv') # reading the dataset
    cv = CountVectorizer()
    countMatrix = cv.fit_transform(data['comb'])
    similarity = cosine_similarity(countMatrix) # creating the similarity matrix
    return (data, similarity)

def Recommend(movie):
    movie = movie.lower()
    try:
        data.head()
        similarity.shape
    except:
        (data, similarity) = createSimilarity()
    if movie not in data['movie_title'].unique():
        return 'Désolé ce film ne se trouve pas dans notre base de données.. essayez un autre!'
    else:
        movieIndex = data.loc[data['movie_title'] == movie].index[0]
        lst = list(enumerate(similarity[movieIndex]))
        lst = sorted(lst, key=lambda x: x[1], reverse=True)
        lst = lst[1:20] 
        movieList = []
        for i in range(len(lst)):
            a = lst[i][0]
            movieList.append(data['movie_title'][a])
        return movieList